<a href="https://colab.research.google.com/github/poziryna84/Dr_Cervantes_AI_Saturday_Madrid/blob/master/Unificacion_tablones_medical_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import xml.etree.ElementTree as et 
import pandas as pd
import re

## Carga y limpieza basica de los datos.


In [0]:
def path_to_df(path):
  '''
  Function that takes a path of a xml file and returns pandas data frame object.
  '''
  xtree = et.parse(path)
  xroot = xtree.getroot()
  lista_1 = []
  for i in xroot:
    lista_1.append(i)
  etiquetas = []
  valores = []
  for i in lista_1:
    valores_ = []
    for e in i:
      valores_.append(e.text)
      if e.tag not in etiquetas:
        etiquetas.append(e.tag)
    valores.append(valores_)
  df_out = pd.DataFrame(valores)
  df_out.columns = etiquetas
  return df_out
  

In [0]:
df_raw = path_to_df('/content/drive/My Drive/Datos_medicos/ALTAS-SEMANHIS.xml')

In [0]:
df_raw = df_raw.drop_duplicates(keep='first')

In [0]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19467 entries, 0 to 20230
Data columns (total 18 columns):
IdIngreso                       19467 non-null object
N_x00BA_HISTORIAL               19467 non-null object
FECHA_ALTA                      19467 non-null object
SERVICIO                        19467 non-null object
MOTIVO_CONSULTA                 19460 non-null object
ENF_ACTUAL                      19460 non-null object
EXPLORACION1                    19399 non-null object
ANALITICA1                      19399 non-null object
EXPL_DIAGNOSTICAS1              19399 non-null object
INTERVENCIONES1                 19399 non-null object
EvolucionComentarios            19396 non-null object
MotivoAlta                      19396 non-null object
DiagnosticoPrincipalAlta        18656 non-null object
ServicioAlta                    18649 non-null object
DestinoAlta1                    18628 non-null object
RecomendacionesTerapeuticas1    18414 non-null object
CODIGODIAG1                

In [0]:
df_raw = df_raw[df_raw['DIAG1'].notna()]

In [0]:
df_raw = df_raw.drop_duplicates(subset = ['IdIngreso'], keep ='first')

## Creamos nueva columna 'aux_codigo' convertiendo los codigos de diagnosticos en columna 'CODIGODIAG1' en tipo float.

In [0]:
df_raw['aux_codigo'] = df_raw['CODIGODIAG1'].apply(lambda x: float(x) 
if not x.lower().startswith('v') else 0.0)

# Cargamos el fichero .txt con los codigos y sus diagnosticos.

In [0]:
lines = []
for line in open('/content/drive/My Drive/Datos_medicos/CIE9_mapeoPorGrupos (2).txt', encoding = "ISO-8859-1"):
  if (line != '\n') and line != ('CLASIFICACION DE ENFERMEDADES Y LESIONES\n'):
    lines.append((str(line)))

In [0]:
lista_general = []
lista_specific = []
for i in lines:
  if i.replace('.', '').split()[0].isdigit():
    lista_general.append(i)
    lista_specific.append('')
  else:
    lista_specific.append(i)
    lista_general.extend([lista_general[-1]])

### Creamos data frame df_aux con los diagnosticos y los codigos.

In [0]:
df_aux = pd.DataFrame([lista_general, lista_specific]).T
df_aux.columns = ['general', 'specific']

In [0]:
df_aux = df_aux[df_aux['specific'] != '']

In [0]:
df_aux['min_specific'] = df_aux['specific'].apply(lambda x: int(re.findall('(\d+)', x)[0]))


In [0]:
df_aux['max_specific'] = df_aux['specific'].apply(lambda x: int(re.findall('(\d+)', x)[1]) if 
                                                  len(re.findall('(\d+)', x)) == 2 
                                                      else int(re.findall('(\d+)', x)[0]))

In [0]:
df_aux['minmax_specific'] = list(zip(df_aux.min_specific, df_aux.max_specific))

In [0]:
df_aux['minmax_general'] = df_aux['general'].apply(lambda x: re.findall('(\d+)\-(\d+)', x)[0])

In [0]:
def general(number):
  '''
     Funcion que coge como entrada un numero y devuelve
     un diagnostico general.
  '''
  for i in df_aux['minmax_general'].drop_duplicates():
    if (number >= int(i[0])) and (number <= int(i[1])):
      string = df_aux['general'][df_aux['minmax_general'] == i].values[0]
      output = string.replace('\n', '')
      return output

In [0]:
def specific(number):
  '''
     Funcion que coge como entrada un numero y devuelve
     un diagnostico especifico.
  '''
  for i in df_aux['minmax_specific']:
    if (number >= int(i[0])) and (number <= int(i[1])):
      string = df_aux['specific'][df_aux['minmax_specific'] == i].values[0]
      output = string.replace('\n', '')
      return output

# Creamos nuevos variables 'GENERAL_DIAG' y 'SPECIFIC_DIAG' identificando los diagnosticos generales y especificos por la columna 'aux_codigo' aplicando las funciones general y specific.

In [0]:
df_raw['GENERAL_DIAG'] = df_raw['aux_codigo'].apply(lambda x: general(x))

In [0]:
df_raw['GENERAL_DIAG'].value_counts(dropna = False)

8. ENFERMEDADES DEL APARATO RESPIRATORIO (460-519)                                                  2039
13. ENFERMEDADES DEL SISTEMA OSTEO-MIOARTICULAR Y TEJIDO CONJUNTIVO (710-739)                       1994
7. ENFERMEDADES DEL SISTEMA CIRCULATORIO (390-459)                                                  1708
9. ENFERMEDADES DEL APARATO DIGESTIVO (520-579)                                                     1354
17. LESIONES Y ENVENENAMIENTOS (800-999)                                                            1209
2. NEOPLASIAS (140-239)                                                                             1030
10. ENFERMEDADES DEL APARATO GENITOURINARIO (580-629)                                               1015
16. SÍNTOMAS, SIGNOS Y ESTADOS MAL DEFINIDOS (780-799)                                               897
6. ENFERMEDADES DEL SISTEMA NERVIOSO Y DE LOS ÓRGANOS DE LOS SENTIDOS (320-389)                      673
NaN                                                    

In [0]:
df_raw = df_raw[df_raw['GENERAL_DIAG'].notna()]

In [0]:
df_raw['SPECIFIC_DIAG'] = df_raw['aux_codigo'].apply(lambda x: specific(x))

In [0]:
df_raw['SPECIFIC_DIAG'].value_counts(dropna = False)

OTRAS FORMAS DE ENFERMEDAD CARDIACA (420-429)                           864
NaN                                                                     776
ARTROPATÍAS Y TRASTORNOS RELACIONADAS (710-719)                         766
SÍNTOMAS (780-789)                                                      690
FRACTURAS (800-829)                                                     623
                                                                       ... 
ENFERMEDADES BACTERIANAS ZOONOSICAS (020-027)                             1
LESION SUPERFICIAL (910-919)                                              1
HELMINTIASIS (120-129)                                                    1
COMPLICACIONES PRINCIPALMENTE RELACIONADAS CON EL EMBARAZO (640-649)      1
LESIÓN POR APLASTAMIENTO (925-929)                                        1
Name: SPECIFIC_DIAG, Length: 88, dtype: int64

In [0]:
df_raw.to_excel('/content/drive/My Drive/Datos_medicos/datos_medicos.xlsx')